In [1]:
import ahrs
%load_ext autoreload
%autoreload 2

In [3]:
import torch
import numpy as np
from torch import nn
from torch.utils.data import DataLoader
import os

os.chdir("../")

print("cuda: %s" % torch.cuda.is_available())

from src.dataset import EuRoC
from src.networks import IMUNet
from src.losses import IMULoss
from src.metrics import (
    metric_aoe_test,
    metric_aoe_training,
    metric_ave_test,
    metric_ave_training,
    metric_aye_test,
    metric_rte_improvement_test,
    metric_rte_test,
)
from src.training import train_model
from src.test import test_model

cuda: True


## EuRoC


In [4]:
T = 64
training_samples = 16000 // (T * 2) * (T * 2)
train_seqs = [
    "MH_01_easy",
    "MH_03_medium",
    "MH_05_difficult",
    "V1_02_medium",
    "V2_01_easy",
    "V2_03_difficult",
]
test_seqs = ["MH_02_easy", "MH_04_difficult", "V1_03_difficult", "V2_02_medium"]

euroc = EuRoC(
    "original_datasets/euroc",
    "datasets/euroc",
    train_seqs,
    test_seqs,
    training_samples,
    T,
)

In [5]:
from src.lie_algebra import SO3
from src.lie_algebra import rad2deg, deg2rad
from src.utils import btmm
from src.metrics import RMSE

In [6]:
def quaternion_distance(q1, q2):
    # Compute the cosine of the half-angle between the two quaternions
    cos_half_angle = np.abs(np.einsum("ij,ij->i", q1, q2))

    # Compute the half-angle itself
    cos_half_angle = np.clip(cos_half_angle, -1, 1)
    half_angle = np.arccos(cos_half_angle)

    # The angle between the two quaternions is twice this half-angle
    angle = 2 * half_angle

    return angle

#### Madgwick


In [7]:
for seq in train_seqs + test_seqs:
    print(seq)
    gyro, acc = (
        euroc.data[seq]["imu"][:, :3].cpu().numpy(),
        euroc.data[seq]["imu"][:, 3:].cpu().numpy(),
    )
    rots = euroc.data[seq]["rots"].cpu().numpy()
    qs = euroc.data[seq]["qs"].cpu().numpy()[1:]
    q0 = euroc.data[seq]["qs"].cpu().numpy()[0]
    attitude = ahrs.filters.Madgwick(acc=acc, gyr=gyro, frequency=200, q0=q0)
    print(
        "quaternion distance error (deg): ",
        rad2deg(quaternion_distance(attitude.Q, qs).mean()),
    )
    print(
        "AOE (deg): ",
        RMSE(
            rad2deg(
                (
                    SO3.log(
                        btmm(
                            SO3.from_quaternion(torch.tensor(attitude.Q))[:-200],
                            torch.tensor(rots[1:]).double()[:-200],
                        )
                    )
                )
            )
        ),
    )
    print(
        "AOE (deg): ",
        RMSE(
            rad2deg(
                (
                    SO3.log(
                        btmm(
                            SO3.from_quaternion(torch.tensor(attitude.Q))[:-200],
                            torch.tensor(rots[1:]).double()[:-200],
                        )
                    )
                )
            ),
            report_individual_axis=True,
        ),
    )
    print("--------------------")

MH_01_easy
quaternion distance error (deg):  96.11123444798035
AOE (deg):  tensor(106.8376, dtype=torch.float64)
AOE (deg):  tensor([94.6556, 23.8680, 43.4157], dtype=torch.float64)
--------------------
MH_03_medium
quaternion distance error (deg):  104.18637478738286
AOE (deg):  tensor(116.5117, dtype=torch.float64)
AOE (deg):  tensor([105.1989,  25.4674,  43.1229], dtype=torch.float64)
--------------------
MH_05_difficult
quaternion distance error (deg):  97.9688528297474
AOE (deg):  tensor(110.7761, dtype=torch.float64)
AOE (deg):  tensor([97.9441, 25.1414, 45.2351], dtype=torch.float64)
--------------------
V1_02_medium
quaternion distance error (deg):  68.52569047412186
AOE (deg):  tensor(76.8073, dtype=torch.float64)
AOE (deg):  tensor([69.3600, 10.8528, 31.1572], dtype=torch.float64)
--------------------
V2_01_easy
quaternion distance error (deg):  100.00828334353507
AOE (deg):  tensor(111.9202, dtype=torch.float64)
AOE (deg):  tensor([100.4679,  25.8594,  41.9957], dtype=torch.

#### Complementary


In [8]:
for seq in train_seqs + test_seqs:
    print(seq)
    gyro, acc = (
        euroc.data[seq]["imu"][:, :3].cpu().numpy(),
        euroc.data[seq]["imu"][:, 3:].cpu().numpy(),
    )
    rots = euroc.data[seq]["rots"].cpu().numpy()
    qs = euroc.data[seq]["qs"].cpu().numpy()[1:]
    q0 = euroc.data[seq]["qs"].cpu().numpy()[0]
    attitude = ahrs.filters.Complementary(acc=acc, gyr=gyro, frequency=200, q0=q0)
    print(
        "quaternion distance error (deg): ",
        rad2deg(quaternion_distance(attitude.Q, qs).mean()),
    )
    print(
        "AOE (deg): ",
        RMSE(
            rad2deg(
                (
                    SO3.log(
                        btmm(
                            SO3.from_quaternion(torch.tensor(attitude.Q))[:-200],
                            torch.tensor(rots[1:]).double()[:-200],
                        )
                    )
                )
            )
        ),
    )
    print(
        "AOE (deg): ",
        RMSE(
            rad2deg(
                (
                    SO3.log(
                        btmm(
                            SO3.from_quaternion(torch.tensor(attitude.Q))[:-200],
                            torch.tensor(rots[1:]).double()[:-200],
                        )
                    )
                )
            ),
            report_individual_axis=True,
        ),
    )
    print("--------------------")

MH_01_easy
quaternion distance error (deg):  110.23311660064168
AOE (deg):  tensor(119.4322, dtype=torch.float64)
AOE (deg):  tensor([109.6874,  24.6514,  40.3116], dtype=torch.float64)
--------------------
MH_03_medium
quaternion distance error (deg):  95.67301070519203
AOE (deg):  tensor(108.3226, dtype=torch.float64)
AOE (deg):  tensor([102.4391,  18.8187,  29.7636], dtype=torch.float64)
--------------------
MH_05_difficult
quaternion distance error (deg):  82.52119834824325
AOE (deg):  tensor(95.5110, dtype=torch.float64)
AOE (deg):  tensor([92.2366, 12.8580, 21.1996], dtype=torch.float64)
--------------------
V1_02_medium
quaternion distance error (deg):  86.51696634697032
AOE (deg):  tensor(100.0504, dtype=torch.float64)
AOE (deg):  tensor([94.9904, 20.6411, 23.6822], dtype=torch.float64)
--------------------
V2_01_easy
quaternion distance error (deg):  110.18835055109467
AOE (deg):  tensor(118.8946, dtype=torch.float64)
AOE (deg):  tensor([114.6748,  24.5341,  19.5879], dtype=to

## TUMVI (Madgwick)


### TUMVICali


In [9]:
from src.dataset import TUMVICali

T = 64
training_samples = 16000 // (T * 2) * (T * 2)
train_seqs = [
    "dataset-room1_512_16",
    "dataset-room3_512_16",
    "dataset-room5_512_16",
]
test_seqs = ["dataset-room2_512_16", "dataset-room4_512_16", "dataset-room6_512_16"]

tumvicali = TUMVICali(
    "/home/kksix/datasets/tumvi",
    "datasets/tumvicali",
    train_seqs,
    test_seqs,
    training_samples,
    T,
)

In [10]:
for seq in train_seqs + test_seqs:
    print(seq)
    gyro, acc = (
        tumvicali.data[seq]["imu"][:, :3].cpu().numpy(),
        tumvicali.data[seq]["imu"][:, 3:].cpu().numpy(),
    )
    rots = tumvicali.data[seq]["rots"].cpu().numpy()
    qs = tumvicali.data[seq]["qs"].cpu().numpy()[1:]
    q0 = tumvicali.data[seq]["qs"].cpu().numpy()[0]
    attitude = ahrs.filters.Madgwick(acc=acc, gyr=gyro, frequency=200, q0=q0)
    print(
        "quaternion distance error (deg): ",
        rad2deg(quaternion_distance(attitude.Q, qs).mean()),
    )
    print(
        "AOE (deg): ",
        RMSE(
            rad2deg(
                (
                    SO3.log(
                        btmm(
                            SO3.from_quaternion(torch.tensor(attitude.Q))[:-200],
                            torch.tensor(rots[1:]).double()[:-200],
                        )
                    )
                )
            )
        ),
    )
    print(
        "AOE (deg): ",
        RMSE(
            rad2deg(
                (
                    SO3.log(
                        btmm(
                            SO3.from_quaternion(torch.tensor(attitude.Q))[:-200],
                            torch.tensor(rots[1:]).double()[:-200],
                        )
                    )
                )
            ),
            report_individual_axis=True,
        ),
    )
    print("--------------------")

dataset-room1_512_16
quaternion distance error (deg):  6.493920963958345
AOE (deg):  tensor(7.6052, dtype=torch.float64)
AOE (deg):  tensor([1.8835, 2.4881, 6.9355], dtype=torch.float64)
--------------------
dataset-room3_512_16
quaternion distance error (deg):  5.282498404312826
AOE (deg):  tensor(6.0291, dtype=torch.float64)
AOE (deg):  tensor([1.7691, 2.6030, 5.1424], dtype=torch.float64)
--------------------
dataset-room5_512_16
quaternion distance error (deg):  3.3626720217586636
AOE (deg):  tensor(3.8289, dtype=torch.float64)
AOE (deg):  tensor([1.4227, 1.5812, 3.1837], dtype=torch.float64)
--------------------
dataset-room2_512_16
quaternion distance error (deg):  11.942508551533317
AOE (deg):  tensor(14.4489, dtype=torch.float64)
AOE (deg):  tensor([ 2.4062,  3.6125, 13.7815], dtype=torch.float64)
--------------------
dataset-room4_512_16
quaternion distance error (deg):  3.34162751171322
AOE (deg):  tensor(3.5499, dtype=torch.float64)
AOE (deg):  tensor([1.1443, 1.9262, 2.7536

### TUMVIUncali


In [13]:
from src.dataset import TUMVIUncali

T = 64
training_samples = 16000 // (T * 2) * (T * 2)
train_seqs = [
    "dataset-room1_512_16",
    "dataset-room3_512_16",
    "dataset-room5_512_16",
]
test_seqs = ["dataset-room2_512_16", "dataset-room4_512_16", "dataset-room6_512_16"]

tumviuncali = TUMVIUncali(
    "/home/kksix/datasets/tumvi",
    "datasets/tumviuncali",
    train_seqs,
    test_seqs,
    training_samples,
    T,
)

In [14]:
for seq in train_seqs + test_seqs:
    print(seq)
    gyro, acc = (
        tumviuncali.data[seq]["imu"][:, :3].cpu().numpy(),
        tumviuncali.data[seq]["imu"][:, 3:].cpu().numpy(),
    )
    rots = tumviuncali.data[seq]["rots"].cpu().numpy()
    qs = tumviuncali.data[seq]["qs"].cpu().numpy()[1:]
    q0 = tumviuncali.data[seq]["qs"].cpu().numpy()[0]
    attitude = ahrs.filters.Madgwick(acc=acc, gyr=gyro, frequency=200, q0=q0)
    print(
        "quaternion distance error (deg): ",
        rad2deg(quaternion_distance(attitude.Q, qs).mean()),
    )
    print(
        "AOE (deg): ",
        RMSE(
            rad2deg(
                (
                    SO3.log(
                        btmm(
                            SO3.from_quaternion(torch.tensor(attitude.Q))[:-200],
                            torch.tensor(rots[1:]).double()[:-200],
                        )
                    )
                )
            )
        ),
    )
    print(
        "AOE (deg): ",
        RMSE(
            rad2deg(
                (
                    SO3.log(
                        btmm(
                            SO3.from_quaternion(torch.tensor(attitude.Q))[:-200],
                            torch.tensor(rots[1:]).double()[:-200],
                        )
                    )
                )
            ),
            report_individual_axis=True,
        ),
    )
    print("--------------------")

dataset-room1_512_16
quaternion distance error (deg):  64.28535374289159
AOE (deg):  tensor(71.1809, dtype=torch.float64)
AOE (deg):  tensor([13.2552, 16.7123, 67.9097], dtype=torch.float64)
--------------------
dataset-room3_512_16
quaternion distance error (deg):  71.63311738755718
AOE (deg):  tensor(84.1534, dtype=torch.float64)
AOE (deg):  tensor([19.9969, 12.1909, 80.8288], dtype=torch.float64)
--------------------
dataset-room5_512_16
quaternion distance error (deg):  84.73408906456628
AOE (deg):  tensor(95.7144, dtype=torch.float64)
AOE (deg):  tensor([26.4420, 24.5851, 88.6433], dtype=torch.float64)
--------------------
dataset-room2_512_16
quaternion distance error (deg):  103.07850731880073
AOE (deg):  tensor(115.4424, dtype=torch.float64)
AOE (deg):  tensor([ 12.5530,  14.1258, 113.8852], dtype=torch.float64)
--------------------
dataset-room4_512_16
quaternion distance error (deg):  51.80132229965813
AOE (deg):  tensor(59.1102, dtype=torch.float64)
AOE (deg):  tensor([12.74